<a href="https://colab.research.google.com/github/lauralpezb/use-sql-with-python-tutorial/blob/main/sql_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primeros pasos de SQL en Python, dominando los datos
## Por Valen y Lau - DataSoul Podcast
### Astrocuentos

## Iniciemos con SQLite3

In [1]:
# Importar librerías
import pandas as pd
import sqlite3

In [2]:
# Crear una conexión a la base de datos en memoria (esto simula una base de datos SQL)
conn = sqlite3.connect(":memory:")

In [ ]:
# Datos de ejemplo: ventas de una tienda
data = {
    'producto': ['Manzana', 'Leche', 'Bistec', 'Yogurt', 'Jabon', 'Repollo', 'CocaCola', 'Pizza', 'Chocolatina', 'Cerveza'],
    'categoria': ['Frutas', 'Lacteos', 'Carnes', 'Lacteos', 'Limpieza', 'Verduras', 'Bebidas', 'Congelados', 'Dulces', 'Bebidas'],
    'precio': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550],
    'cantidad': [10, 15, 20, 25, 30, 35, 40, 45, 50, 55],
    'fecha_venta': pd.date_range('2023-01-01', periods=10, freq='D')
}

df = pd.DataFrame(data)

df.head()

In [ ]:
# Cargar los datos a una tabla SQL en SQLite
df.to_sql('ventas', conn, index=False, if_exists='replace')

In [5]:
# Uso básico de SQL con Pandas y SQLite
# Realizar una consulta sencilla: obtener todas las ventas
query = "SELECT * FROM ventas"
ventas_df = pd.read_sql(query, conn)
ventas_df

,producto,categoria,precio,cantidad,fecha_venta
0,Manzana,Frutas,100,10,2023-01-01 00:00:00
1,Leche,Lacteos,150,15,2023-01-02 00:00:00
2,Bistec,Carnes,200,20,2023-01-03 00:00:00
3,Yogurt,Lacteos,250,25,2023-01-04 00:00:00
4,Jabon,Limpieza,300,30,2023-01-05 00:00:00
5,Repollo,Verduras,350,35,2023-01-06 00:00:00
6,CocaCola,Bebidas,400,40,2023-01-07 00:00:00
7,Pizza,Congelados,450,45,2023-01-08 00:00:00
8,Chocolatina,Dulces,500,50,2023-01-09 00:00:00
9,Cerveza,Bebidas,550,55,2023-01-10 00:00:00


In [6]:
# Construcción de consultas: Seleccionar columnas y filtros
# Queremos saber el precio de los productos con cantidad superior a 20
query_bajo = "SELECT producto, precio FROM ventas WHERE cantidad > 20"
ventas_bajo = pd.read_sql(query_bajo, conn)
ventas_bajo

,producto,precio
0,Yogurt,250
1,Jabon,300
2,Repollo,350
3,CocaCola,400
4,Pizza,450
5,Chocolatina,500
6,Cerveza,550


In [7]:
# Uso de Agregaciones
# ¿Cuál es el precio promedio de los productos?
query_promedio = "SELECT AVG(precio) AS precio_promedio FROM ventas"
promedio_df = pd.read_sql(query_promedio, conn)
promedio_df

,precio_promedio
0,325.0


In [8]:
# Tenemos diferentes categorías y queremos saber cuántos productos hay disponibles
query_agrupado = """
SELECT categoria, SUM(cantidad) AS cantidad
FROM ventas
GROUP BY categoria
"""
agrupado_df = pd.read_sql(query_agrupado, conn)
agrupado_df

,categoria,cantidad
0,Bebidas,95
1,Carnes,20
2,Congelados,45
3,Dulces,50
4,Frutas,10
5,Lacteos,40
6,Limpieza,30
7,Verduras,35


In [9]:
# Join entre tablas
data_productos = {
    'producto': ['Manzana', 'Leche', 'Bistec', 'Yogurt', 'Jabon', 'Repollo', 'CocaCola', 'Pizza', 'Chocolatina', 'Cerveza'],
    'zona': ['A', 'B', 'C', 'B', 'D', 'A', 'B', 'B', 'E', 'F']
}

df_productos = pd.DataFrame(data_productos)

# Crear la tabla de productos
df_productos.to_sql('productos', conn, index=False, if_exists='replace')

# Realizar un JOIN entre las tablas 'ventas' y 'productos' para obtener la zona del producto junto con las ventas
query_join = """
SELECT v.producto, v.precio, v.cantidad, p.zona
FROM ventas v
JOIN productos p ON v.producto = p.producto
"""
join_df = pd.read_sql(query_join, conn)
join_df

,producto,precio,cantidad,zona
0,Manzana,100,10,A
1,Leche,150,15,B
2,Bistec,200,20,C
3,Yogurt,250,25,B
4,Jabon,300,30,D
5,Repollo,350,35,A
6,CocaCola,400,40,B
7,Pizza,450,45,B
8,Chocolatina,500,50,E
9,Cerveza,550,55,F


In [10]:
# Subconsultas
# Queremos encontrar los productos cuyo precio es mayor al promedio de todos los productos
query_subconsulta = """
SELECT producto, precio
FROM ventas
WHERE precio > (SELECT AVG(precio) FROM ventas)
"""
subconsulta_df = pd.read_sql(query_subconsulta, conn)
subconsulta_df

,producto,precio
0,Repollo,350
1,CocaCola,400
2,Pizza,450
3,Chocolatina,500
4,Cerveza,550


In [11]:
# Mostrar solo los 5 primeros productos más caros
query_limit = "SELECT * FROM ventas ORDER BY precio DESC LIMIT 5"
limit_df = pd.read_sql(query_limit, conn)
limit_df

,producto,categoria,precio,cantidad,fecha_venta
0,Cerveza,Bebidas,550,55,2023-01-10 00:00:00
1,Chocolatina,Dulces,500,50,2023-01-09 00:00:00
2,Pizza,Congelados,450,45,2023-01-08 00:00:00
3,CocaCola,Bebidas,400,40,2023-01-07 00:00:00
4,Repollo,Verduras,350,35,2023-01-06 00:00:00


In [12]:
# Uso de CTE (Common Table Expressions)
# Queremos encontrar los productos con el precio más alto
query_cte = """
WITH PrecioPromedio AS (
    SELECT AVG(precio) AS promedio_precio FROM ventas
)
SELECT producto, precio
FROM ventas, PrecioPromedio
WHERE precio > PrecioPromedio.promedio_precio
"""
cte_df = pd.read_sql(query_cte, conn)
cte_df

,producto,precio
0,Repollo,350
1,CocaCola,400
2,Pizza,450
3,Chocolatina,500
4,Cerveza,550


In [13]:
# Funciones de ventana (Window Functions)
# Queremos calcular el "ranking" de los productos por precio
query_window = """
SELECT producto, precio,
       RANK() OVER (ORDER BY precio DESC) AS ranking_precio
FROM ventas
"""
window_df = pd.read_sql(query_window, conn)
window_df

,producto,precio,ranking_precio
0,Cerveza,550,1
1,Chocolatina,500,2
2,Pizza,450,3
3,CocaCola,400,4
4,Repollo,350,5
5,Jabon,300,6
6,Yogurt,250,7
7,Bistec,200,8
8,Leche,150,9
9,Manzana,100,10


In [14]:
# Uso de particiones con funciones de ventana (Window Functions con PARTITION BY)
# Queremos calcular el "ranking" de los productos dentro de cada zona
query_partition = """
SELECT v.producto, v.precio, p.zona,
       RANK() OVER (PARTITION BY p.zona ORDER BY v.precio DESC) AS ranking_categoria
FROM ventas v
JOIN productos p ON v.producto = p.producto
"""
partition_df = pd.read_sql(query_partition, conn)
partition_df

,producto,precio,zona,ranking_categoria
0,Repollo,350,A,1
1,Manzana,100,A,2
2,Pizza,450,B,1
3,CocaCola,400,B,2
4,Yogurt,250,B,3
5,Leche,150,B,4
6,Bistec,200,C,1
7,Jabon,300,D,1
8,Chocolatina,500,E,1
9,Cerveza,550,F,1


In [15]:
# Consultas con GROUP BY (y HAVING)
# Queremos ver la cantidad total vendida por producto, pero solo mostrar productos que hayan vendido más de 40 unidades
query_groupby_having = """
SELECT producto, SUM(cantidad) AS total_vendido
FROM ventas
GROUP BY producto
HAVING total_vendido > 40
"""
groupby_having_df = pd.read_sql(query_groupby_having, conn)
groupby_having_df

,producto,total_vendido
0,Cerveza,55
1,Chocolatina,50
2,Pizza,45


In [16]:
# Cerrar la conexión al final
conn.close()

Además de sqlite3, que es una de las librerías más comunes para trabajar con bases de datos SQL en Python, existen otras librerías que puedes utilizar para interactuar con bases de datos SQL y realizar consultas directamente desde Python.

## SQLAlchemy

SQLAlchemy  permite ejecutar consultas SQL, además proporciona un Object Relational Mapper (ORM), lo que facilita el trabajo con bases de datos mediante clases de Python. También ofrece un Core para trabajar directamente con SQL de manera más tradicional.
	•	Compatible con múltiples bases de datos como PostgreSQL, MySQL, SQLite, Oracle, entre otras.
	•	Permite usar SQL puro o abstraer el acceso mediante objetos Python (ORM).
	•	Soporta transacciones, conexiones, y manejo de sesiones.

## Pandas + SQLAlchemy

Pandas se puede combinar fácilmente con SQLAlchemy para cargar datos desde bases de datos SQL directamente a un DataFrame de pandas, lo que facilita el análisis de datos. Esto es particularmente útil cuando quieres hacer análisis de datos sobre bases de datos sin tener que escribir demasiado SQL manualmente.
	•	Facilita la integración entre bases de datos SQL y análisis de datos en pandas.
	•	Puedes usar read_sql y to_sql con bases de datos SQL como PostgreSQL, MySQL, SQLite, entre otras.


In [17]:
# Importar librerías
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

In [18]:
# Crear una base de datos en memoria
DATABASE_URL = "sqlite:///example.db"
engine = create_engine(DATABASE_URL, echo=True)

# Crear una clase base para los modelos
Base = declarative_base()

In [19]:
# Definir un modelo de ejemplo
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    email = Column(String, nullable=False)

    def __repr__(self):
        return f"<User(id={self.id}, name={self.name}, email={self.email})>"

In [ ]:
# Crear las tablas en la base de datos
Base.metadata.create_all(engine)

In [21]:
# Crear una sesión para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# CRUD: Crear
new_user = User(name="Juan Pérez", email="juan.perez@example.com")
session.add(new_user)
session.commit()
print(f"Usuario creado: {new_user}")

In [ ]:
# CRUD: Leer
users = session.query(User).all()
print("Usuarios en la base de datos:")
for user in users:
    print(user)

In [ ]:
# CRUD: Actualizar
user_to_update = session.query(User).filter_by(name="Juan Pérez").first()
if user_to_update:
    user_to_update.email = "nuevo.email@example.com"
    session.commit()
    print(f"Usuario actualizado: {user_to_update}")

In [ ]:
# CRUD: Eliminar
user_to_delete = session.query(User).filter_by(name="Juan Pérez").first()
if user_to_delete:
    session.delete(user_to_delete)
    session.commit()
    print(f"Usuario eliminado: {user_to_delete}")

In [ ]:
# Verificar usuarios después de eliminación
users = session.query(User).all()
print("Usuarios restantes en la base de datos:")
for user in users:
    print(user)

Resumen de librerías para SQL en Python:

	•	SQLAlchemy: ORM completo, flexible y compatible con varias bases de datos.
	•	Psycopg2: Para trabajar con bases de datos PostgreSQL.
	•	MySQL Connector: Para trabajar con bases de datos MySQL.
	•	Pandas + SQLAlchemy: Para integrar análisis de datos de SQL directamente en pandas.
	•	Django ORM: ORM de Django, ideal para aplicaciones web.
	•	PyODBC: Para conexiones ODBC a bases de datos.
	•	Peewee: ORM ligero y fácil de usar.
	•	Dask + SQLAlchemy: Para trabajar con grandes volúmenes de datos distribuidos.

Cada librería tiene su propósito y es útil en diferentes escenarios dependiendo de las necesidades de tu proyecto.